# Financial Statements in the OpenBB Platform

OpenBB Platform data extensions provide access to financial statements as quarterly or annual.  There are also endpoints for ratios and other common non-GAAP metrics.  Most data providers require a subscription to access all data. Refer to the website of a specific provider for details on entitlements and coverage.

Financial statement functions are grouped under the `obb.equity.fundamental` module.

## Endpoints

The typical financial statements consist of three endpoints:

- Balance Sheet: `obb.equity.fundamental.balance()`
- Income Statement: `obb.equity.fundamental.income()`
- Cash Flow Statement: `obb.equity.fundamental.cash()`

The main parameters are:

- `symbol`: The company's symbol.
- `period`: 'annual' or 'quarter'. Default is 'annual'.
- `limit`: Limit the number of results returned, from the latest. Default is 5. For perspective, 150 will go back to 1985. The amount of historical records varies by provider.

### Field Names

Some considerations to keep in mind when working with financial statements data are:

- Every data provider has their own way of parsing and organizing the three financial statements.
- Items within each statement will vary by source and by the type of company reporting.
- Names of line items will vary by source.
- "Date" values may differ because they are from the period starting/ending or date of reporting.

This example highlights how different providers will have different labels for compnay facts.


**Note**: API Keys are required for FMP, Intrinio, and Polygon.

In [48]:
import pandas as pd
from openbb import obb

In [49]:
df = pd.DataFrame()

df["yfinance"] = (
    obb.equity.fundamental.balance(
        "TGT", provider="yfinance"
    )  # There is no limit for yFinance, historical data is limited.
    .to_df()
    .get("total_assets")
    .head(3)
)

df["fmp"] = (
    obb.equity.fundamental.balance("TGT", provider="fmp", limit=3)
    .to_df()
    .get("total_assets")
)

df["intrinio"] = (
    obb.equity.fundamental.balance("TGT", provider="intrinio", limit=3)
    .to_df()
    .get("total_assets")
)

df["polygon"] = (
    obb.equity.fundamental.balance("TGT", provider="polygon", limit=3)
    .to_df()
    .get("total_assets")
)

df

,yfinance,fmp,intrinio,polygon
0,5.535600e+10,5.535600e+10,5.535600e+10,5.535600e+10
1,5.333500e+10,5.333500e+10,5.333500e+10,5.333500e+10
2,5.381100e+10,5.381100e+10,5.381100e+10,5.381100e+10


### Weighted Average Shares Outstanding

This key metric will be found under the income statement.  It might also be called, 'basic', and the numbers do not include authorized but unissued shares.  A declining count over time is a sign that the company is returning capital to shareholders in the form of buy backs.  Under ideal circumstances, it is more capital-efficient, for both company and shareholders, because distributions are double-taxed.  The company pays income tax on paid dividends, and the beneficiary pays income tax again on receipt.

A company will disclose how many shares are outstanding at the end of the period  as a weighted average over the reporting period - three months.

Let's take a look at Target.  To make the numbers easier to read, we'll divide the entire column by one million.

In [50]:
data = obb.equity.fundamental.income(
    "TGT", provider="fmp", limit=150, period="quarter"
).to_df()

shares = data["weighted_average_basic_shares_outstanding"] / 1000000

display(shares.head(1))

display(shares.tail(1))

0    462.5
Name: weighted_average_basic_shares_outstanding, dtype: float64

149    1169.248
Name: weighted_average_basic_shares_outstanding, dtype: float64

Thirty-seven years later, the share count is approaching a two-thirds reduction.  12.2% over the past five years.  In four reporting periods, 1.3 million shares have been taken out of the float.

In [51]:
display(shares.pct_change(20).iloc[-1])

display(shares.iloc[-4] - shares.iloc[-1])

0.3362834285714287

-65.75199999999995

With an average closing price of $143.37, that represents approximately $190M in buy backs.

In [52]:
price = obb.equity.price.historical(
    "TGT", start_date="2022-10-01", provider="fmp"
).to_df()

round((price["close"].mean() * 1300000) / 1000000, 2)

190.75

### Dividends Paid

Dividends paid is in the cash flow statement.  We can calculate the amount-per-share with the reported data.

In [54]:
dividends = obb.equity.fundamental.cash(
    "TGT", provider="fmp", limit=150, period="quarter"
).to_df()[["payment_of_dividends"]]

dividends["shares"] = data[["weighted_average_basic_shares_outstanding"]]
dividends["div_per_share"] = abs(
    dividends["payment_of_dividends"] / dividends["shares"]
)

dividends["div_per_share"].tail(4)

136    0.040339
137    0.023793
138    0.020690
139    0.022969
Name: div_per_share, dtype: float64

This can be compared against the real amounts paid to common share holders, as announced.  Note that the dates above represent the report date, and that dividends paid are attributed to the quarter they were paid in.  The value from "2023-01-28" equates to the fourth quarter of 2022.

In [55]:
data = obb.equity.fundamental.dividends("TGT", provider="fmp").to_df()[
    ["ex_dividend_date", "amount"]
]
data.ex_dividend_date = data.ex_dividend_date.astype(str)
data.set_index("ex_dividend_date").loc["2023-08-15":"2022-11-15"]

,amount
ex_dividend_date,
2023-08-15,1.10
2023-05-16,1.08
2023-02-14,1.08
2022-11-15,1.08


The numbers check out, and the $2B paid to investors over four quarters is more than ten times the $190M returned through share buy backs.

### Financial Attributes

The `openbb-intrinio` data extension has an endpoint for extracting a single fact from financial statements.  There is a helper function for looking up the correct `tag`.

**Note:** Intrinio does not offer a free API level with access to data.

#### Search Financial Attributes

Search attributes by keyword.

In [10]:
(obb.equity.fundamental.search_attributes("marketcap", provider="intrinio").to_df())

,id,name,tag,statement_code,statement_type,type,unit,parent_name,sequence,factor,transaction
0,tag_BgkbWy,Market Capitalization,marketcap,calculations,industrial,valuation,usd,NaN,NaN,NaN,NaN
1,tag_kylOqz,Market Capitalization,marketcap,calculations,financial,valuation,usd,NaN,NaN,NaN,NaN
2,tag_XLRlqy,Market Sector,market_sector,current,NaN,security,string,NaN,NaN,NaN,NaN
3,tag_2gBA8y,Market Category,market_category,current,NaN,security,string,NaN,NaN,NaN,NaN
4,tag_DzonXe,Marketing Expense,marketingexpense,income_statement,industrial,income_statement_metric,usd,totaloperatingexpenses,9.0,+,debit
...,...,...,...,...,...,...,...,...,...,...,...
95,tag_nzJAmX,Total Long-Term Debt,ltdebtandcapleases,calculations,financial,metric,usd,NaN,NaN,NaN,NaN
96,tag_9XaL5g,Other Net Changes in Cash,othernetchangesincash,cash_flow_statement,industrial,cash_flow_statement_metric,usd,netchangeincash,33.0,+,debit
97,tag_5X7p6z,Other Net Changes in Cash,othernetchangesincash,cash_flow_statement,financial,cash_flow_statement_metric,usd,netchangeincash,37.0,+,debit
98,tag_qzEwng,"Changes in Operating Assets and Liabilities, net",increasedecreaseinoperatingcapital,cash_flow_statement,financial,cash_flow_statement_metric,usd,netcashfromcontinuingoperatingactivities,8.0,+,debit


The `tag` is what we need, in this case it is what we searched for.

In [20]:
marketcap = obb.equity.fundamental.historical_attributes(
    symbol="TGT", tag="marketcap", frequency="quarterly", provider="intrinio"
).to_df()

marketcap.tail(5)

,symbol,tag,value
date,,,
2023-09-30,TGT,marketcap,4.951153e+10
2023-12-31,TGT,marketcap,6.443403e+10
2024-03-31,TGT,marketcap,8.082004e+10
2024-06-30,TGT,marketcap,6.814283e+10
2024-08-22,TGT,marketcap,7.387608e+10


Doing some quick math, and ignoring the most recent value, we can see that the market cap of Target was down nearly a quarter over the last four reporting periods.

In [40]:
marketcap.index = marketcap.index.astype(str)
(
    (marketcap.loc["2023-09-30"].value - marketcap.loc["2022-12-31"].value)
    / marketcap.loc["2022-12-31"].value
)

-0.243767327909974

Historial market cap is also available as a daily metric from FMP. We can resample it as quarterly to approximate the same results.

In [43]:
df = obb.equity.historical_market_cap(
    "TGT", start_date="2022-01-01", provider="fmp"
).to_df()

resampled = df.copy()
resampled.index = pd.to_datetime(resampled.index)
resampled = resampled[["market_cap"]]
resampled = resampled.resample("QE").last()
resampled.index = resampled.index.astype(str)
display(resampled)
(
    (resampled.loc["2023-09-30"] - resampled.loc["2022-12-31"])
    / resampled.loc["2022-12-31"]
)

,market_cap
date,
2022-03-31,98470080000
2022-06-30,65177644999
2022-09-30,68303916999
2022-12-31,68603112000
2023-03-31,76338867000
2023-06-30,60885040000
2023-09-30,51039112000
2023-12-31,65755313999
2024-03-31,81906462000


market_cap   -0.256023
dtype: float64

## Ratios and Other Metrics

Other valuation functions are derivatives of the financial statements, but the data provider does the math.  Values are typically ratios between line items, on a per-share basis, or as a percent growth.

This data set is where you can find EPS, FCF, P/B, EBIT, quick ratio, etc.

### Quick Ratio

Target's quick ratio could be one reason why its share price is losing traction against the market.  Its ability to pay current obligations is not optimistically reflected in a 0.27 score, approximately 50% below the historical median.

In [56]:
ratios = obb.equity.fundamental.ratios("TGT", limit=50, provider="fmp").to_df()

display(f"Current Quick Ratio: {round(ratios['quick_ratio'].iloc[-1], 4)}")
display(f"Median Quick Ratio: {round(ratios['quick_ratio'].median(), 4)}")

'Current Quick Ratio: 0.8998'

'Median Quick Ratio: 0.6047'

### Free Cash Flow Yield

The `metrics` endpoint, with the `openbb-fmp` data extension, has a field for free cash flow yield.  It is calculated by taking the free cash flow per share divided by the current share price.  We could arrive at this answer by writing some code, but these types of endpoints do the work so we don't have to.  This is part of the value-add that API data distributors provide, they allow you to get straight to work with data.

We'll use this endpoint to extract the data, and compare with some of Target's competition over the last ten years.

In [57]:
# List of other retail chains
tickers = ["COST", "BJ", "DLTR", "DG", "WMT", "BIG", "M", "KSS", "TJX"]

# Create a column for each.
fcf_yield = pd.DataFrame()
for ticker in tickers:
    fcf_yield[ticker] = (
        obb.equity.fundamental.metrics(
            ticker, provider="fmp", period="annual", limit=10
        )
        .to_df()
        .reset_index()
        .set_index("calendar_year")
        .sort_index(ascending=False)["free_cash_flow_yield"]
    )
fcf_yield.transpose()

calendar_year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014
COST,0.027922,0.014860,0.026582,0.039351,0.025906,0.027438,0.060884,0.008941,0.030741,0.037483
BJ,0.029338,0.044709,0.067213,0.113551,0.056631,0.091107,0.026186,0.065871,0.016947,NaN
DLTR,0.018948,0.010756,0.013957,0.075627,0.040338,0.041252,0.034069,0.063465,0.016602,0.041047
DG,0.023149,0.008256,0.037507,0.058973,0.036922,0.046197,0.042609,0.050776,0.039524,0.046052
WMT,0.030577,0.028374,0.065467,0.044595,0.062030,0.057280,0.101023,0.073506,0.059705,NaN
BIG,-1.856996,-0.624151,0.025262,0.115757,0.069464,-0.111853,0.037219,0.100721,0.110443,0.089253
M,0.061077,0.050473,0.270980,0.039111,0.091301,0.101426,0.155761,0.098993,0.065634,0.072322
KSS,0.203512,-0.143961,0.189677,0.147968,0.119492,0.139799,0.096137,0.198790,0.081652,0.110697
TJX,0.027513,0.023498,0.051975,0.039865,0.049788,0.039930,0.053697,0.043328,0.046442,NaN


There are more usage examples on our [website](https://docs.openbb.co/platform/user_guides)